In [5]:
import numpy as np
from dataclasses import dataclass
import re
from pathlib import Path
import ijson
import pandas as pd
import textdistance

In [6]:
filename = "../data/twitter-data-small.json"

In [7]:
@dataclass
class json_handler:
    """
    JSON handler use to handle json input and obtain required information.
    """
    location_dict = {
        "Australian Capital Territory": "act",
        "New South Wales": "nsw",
        "Victoria": "vic",
        "Sydney": "syd",
        "Melbourne": "mel",
        "Hobart": "hob",
        "Brisbane": "bri",
        "Queensland": "qld",
        "Tasmania": "tas",
        
        " - ": " ",
        ", ": " "
    }
    
    @classmethod
    def location(self, string):
        location = string
        for key, value in self.location_dict.items():
            location = re.sub(key, value, location)
        
        # remove duplicate words
        location = re.sub(r'\b(\w+)\b\s+\b\1\b', r'\1', location)
            
        return location.lower().lstrip().rstrip()
    
    @classmethod
    def location_distance(text: str, target: str) -> str:
        score = textdistance.jaro_winkler(text, target)
        return score

In [8]:
@dataclass
class Twitter:
    _id: str = None
    author: str = None
    locat: str = None
        
    def __repr__(self):
        return f"{self._id} | {self.author} | {self.locat}"

In [9]:
FRIST_LINE, START = "item", "start_map"
TWITTER_ID = 'item._id'
AUTHOR_ID = "item.data.author_id"
LOCATION = "item.includes.places.item.full_name"
HAS_VALUE = "string"

In [10]:
def start_json_item(prefix: str, event: str) -> bool:
    return True if prefix == FRIST_LINE and event == START else False
    
def is_twitter_id(prefix: str, event: str, value: any) -> bool:
    return True if prefix == TWITTER_ID and event == HAS_VALUE and value is not None else False

def is_author_id(prefix: str, event: str, value: any) -> bool:
    return True if prefix == AUTHOR_ID and event == HAS_VALUE and value is not None else False

def is_location(prefix: str, event: str, value: any) -> bool:
    return True if prefix == LOCATION and event == HAS_VALUE and value is not None else False

In [15]:
chunk_size = 100000

In [18]:
import ijson.backends

In [33]:
import json

In [40]:
data_size_total = Path(filename).stat().st_size
data_size_total

1471015

In [101]:
data_size_total // 4

367753

In [131]:
with open(filename, mode="r") as f:
    f.seek(367753)
    
    line = f.readline()
    
    chunks = []
    while not re.search("_id", line):
        line = f.readline()
    
    print(line.replace(",", "").replace("\"", "").split(": ")[-1])
    while not re.search("author_id", line):
        line = f.readline()
        
    print(line.replace(",", "").replace("\"", "").split(": ")[-1])
    while not re.search("full_name", line):
        line = f.readline()
    
    print(line.replace(",", "").replace("\"", "").split(": ")[-1])

1412197812243034114

1099659121

Sydney New South Wales

